# Projeto de automação - Pesquisa de preços

## Passo a passo do projeto

1 - Importar e visualizar a base de dados 

2 - Criar o navegador

3 - Para cada item da base de dados:

    3.1 - Procurar o produto no google shopping
    
    3.2 - Verificar se os produtos nos sites estão dentro da faixa de preço
    
4 - Salvar as ofertas que correspondem as condições da pesquisa em um dataframe 

5 - Exportar para excel

6 - Enviar o novo dataframe por email

In [1]:
# importando todas as bibliotecas utilizadas
import pandas as pd
import time
import smtplib
import email.message
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### 1 - Importando e visualizando a base de dados

In [2]:
navegador = webdriver.Chrome()
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,2500,4000
1,rtx 3060,zota galax,2500,5000


#### 2 e 3 - Criando uma função para criar uma instância do navegador e realizar a busca no google

In [3]:
def busca_google(navegador, produto, termos_banidos, preco_min, preco_max):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    lista_ofertas = []
    
    navegador.get("https://www.google.com/")
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    abas_google = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in abas_google:
        if 'Shopping' in item.text:
            item.click()
            break
    time.sleep(7)

    lista_resultados = navegador.find_elements(By.CLASS_NAME, "sh-dgr__content")
    for resultado in lista_resultados:
        try:
            nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
            nome = nome.lower()
            # tratando os nomes do produtos
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True
            tem_todos_termos_produto = True
            for palavra in lista_termos_nome_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False
            if not tem_termos_banidos and tem_todos_termos_produto:
                # tratando os preços dos produtos    
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                preco_min = float(preco_min)
                preco_max = float(preco_max)

                if preco_min <= preco <= preco_max:
                    elemento_ref = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                    elemento_pai = elemento_ref.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
    return lista_ofertas

#### 4 - Criação da tabela de ofertas com os resultados enconstrados

In [4]:
tabela_ofertas = pd.DataFrame()
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']
    
    lista_ofertas_google = busca_google(navegador, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google:
        tabela_ofertas_google = pd.DataFrame(lista_ofertas_google, columns=['Produto', 'Preço', 'Link da Loja'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_ofertas_google])
    else:
        tabela_ofertas_google = None
display(tabela_ofertas)

,Produto,Preço,Link da Loja
0,"iphone 12 apple vermelho, 64gb",3842.10,https://www.google.com/url?url=https://www.ama...
1,"vitrine: iphone 12 apple 64gb azul tela 6,1 câ...",3514.05,https://www.google.com/url?url=https://www.car...
2,iphone 12 64gb - azul - estou zerado,3329.10,https://www.google.com/url?url=https://www.tro...
3,"vitrine: iphone 12 apple 64gb preto tela 6,1 c...",3699.00,https://www.google.com/url?url=https://www.car...
4,iphone 12 64gb | celular apple | usado,3815.00,https://www.google.com/url?url=https://www.enj...
5,"apple iphone 12, 64 gb, verde - totalmente des...",2791.00,https://www.google.com/url?url=https://tiendam...
6,"iphone 12 (seminovo) - 64 gb, branco",3379.00,https://www.google.com/url?url=https://novo.wa...
7,iphone 12 64gb - verde - estou zerado,3329.10,https://www.google.com/url?url=https://www.tro...
8,"smartphone apple iphone 12 - oled 6.1"" 64/128/...",3649.90,https://www.google.com/url?url=https://www.rba...
9,"(seminovo) iphone 12 apple azul, 64gb",3144.15,https://www.google.com/url?url=https://www.taq...


#### 5 - Expotando a tabela para excel

In [5]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

#### 6 - Enviando o e-mail com a tabela de ofertas

In [6]:
def enviar_email():
    corpo_email = f"""
    <p>Prezados, </p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada.</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.</p>
    """
    
    msg = email.message.Message()
    msg['Subject'] = 'Tabela de Ofertas'
    msg['From'] = 'seu e-mail'
    msg['To'] = 'e-mail de quem vai receber'
    password = 'senha gerada pelo google'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )
    
    s=smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    
    # login
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('E-mail enviado')

In [7]:
enviar_email()
navegador.quit()

E-mail enviado
